# Partie 3 – Entraînement du modèle de prédiction

Dans cette partie, nous utilisons le dataset préparé (avec les statistiques
de combat et les caractéristiques des Pokémon) pour entraîner plusieurs modèles
de Machine Learning.

Objectif : prédire le `POURCENTAGE_DE_VICTOIRES` d’un Pokémon à partir de ses
caractéristiques (statistiques de base et génération).


In [ ]:
import pandas as pd

# Chargement du dataset
dataset = pd.read_csv("datas/dataset.csv", delimiter="\t")

# On enlève la colonne d'index inutile si elle existe
if "Unnamed: 0" in dataset.columns:
    dataset = dataset.drop(columns=["Unnamed: 0"])

# On renomme la colonne pour être cohérent avec le reste du code
if "NOMBRE_GENERATIONS" in dataset.columns:
    dataset = dataset.rename(columns={"NOMBRE_GENERATIONS": "GENERATION"})

# important : on enlève les lignes où il manque des valeurs
colonnes_obligatoires = [
    "POINTS_DE_VIE",
    "NIVEAU_ATTAQUE",
    "NIVEAU_DEFENSE",
    "NIVEAU_ATTAQUE_SPECIALE",
    "NIVEAU_DEFENSE_SPECIALE",
    "VITESSE",
    "GENERATION",
    "POURCENTAGE_DE_VICTOIRES",
]

dataset = dataset.dropna(subset=colonnes_obligatoires).reset_index(drop=True)

# Vérification rapide
dataset[colonnes_obligatoires].isna().sum()


Index(['NUMERO', 'NOM', 'TYPE_1', 'TYPE_2', 'POINTS_DE_VIE', 'NIVEAU_ATTAQUE',
       'NIVEAU_DEFENSE', 'NIVEAU_ATTAQUE_SPECIALE', 'NIVEAU_DEFENSE_SPECIALE',
       'VITESSE', 'GENERATION', 'LEGENDAIRE', 'NBR_COMBATS', 'NBR_VICTOIRES',
       'POURCENTAGE_DE_VICTOIRES'],
      dtype='object')

## 1. Définition des variables d'entrée (X) et de la variable cible (y)

Nous utilisons comme variables d'entrée les caractéristiques numériques
du Pokémon (points de vie, attaques, défenses, vitesse, génération).
La variable cible à prédire est le `POURCENTAGE_DE_VICTOIRES`.


In [5]:
# X : données d'entrée (features)
X = dataset[[
    "POINTS_DE_VIE",
    "NIVEAU_ATTAQUE",
    "NIVEAU_DEFENSE",
    "NIVEAU_ATTAQUE_SPECIALE",
    "NIVEAU_DEFENSE_SPECIALE",
    "VITESSE",
    "GENERATION",
]].values

# y : variable à prédire (pourcentage de victoire)
y = dataset["POURCENTAGE_DE_VICTOIRES"].values

X[:5], y[:5]


(array([[ 45,  49,  49,  65,  65,  45,   1],
        [ 60,  62,  63,  80,  80,  60,   1],
        [ 80,  82,  83, 100, 100,  80,   1],
        [ 80, 100, 123, 122, 120,  80,   1],
        [ 39,  52,  43,  60,  50,  65,   1]]),
 array([0.27819549, 0.38016529, 0.67424242, 0.56      , 0.49107143]))

## 2. Séparation du dataset en jeu d'apprentissage et de validation

Nous séparons le dataset en deux parties :

- jeu d'apprentissage (80 %)
- jeu de validation (20 %)

Cela permet d'entraîner le modèle sur une partie des données et
de vérifier ses performances sur des données jamais vues.


In [ ]:
from sklearn.model_selection import train_test_split

X_APPRENTISSAGE, X_VALIDATION, Y_APPRENTISSAGE, Y_VALIDATION = train_test_split(
    X, y, test_size=0.2, random_state=0
)


((640, 7), (160, 7))

## 3. Modèle 1 : Régression linéaire

Nous commençons par un modèle simple de régression linéaire.
Nous entraînons le modèle sur le jeu d'apprentissage et nous
mesurons la précision (R²) sur le jeu de validation.


In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

algorithme = LinearRegression()
algorithme.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)
predictions = algorithme.predict(X_VALIDATION)
precision = r2_score(Y_VALIDATION, predictions)
print(">> ----------  REGRESSION LINEAIRE  ----------")
print(">>  Précision = " + str(precision))
print("----------------------------------------------")


ValueError: Input y contains NaN.

## 4. Modèle 2 : Arbre de décision

Nous essayons maintenant un arbre de décision pour la régression.


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# ----- ALGORITHME 2 : ARBRE DE DÉCISION -----
algorithme = DecisionTreeRegressor()

algorithme.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)
predictions = algorithme.predict(X_VALIDATION)
precision = r2_score(Y_VALIDATION, predictions)

print(">> ----------  ARBRE DE DECISIONS  ----------")
print(">>  Précision = " + str(precision))
print("----------------------------------------------")


## 5. Modèle 3 : Forêt aléatoire (Random Forest)

Enfin, nous testons une forêt aléatoire, souvent plus performante
car elle combine plusieurs arbres de décision.


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# ----- ALGORITHME 3 : FORETS ALEATOIRES -----
algorithme = RandomForestRegressor()

algorithme.fit(X_APPRENTISSAGE, Y_APPRENTISSAGE)
predictions = algorithme.predict(X_VALIDATION)
precision = r2_score(Y_VALIDATION, predictions)

print(">> ----------  FORETS ALEATOIRES  ----------")
print(">>  Précision = " + str(precision))
print("----------------------------------------------")


## 6. Vérification surapprentissage / sous-apprentissage

Nous comparons maintenant la performance sur le jeu d'apprentissage
et sur le jeu de validation pour détecter un éventuel surapprentissage
(overfitting) ou sous-apprentissage (underfitting).


In [ ]:
# Prédictions sur le jeu de validation (avec le dernier modèle entraîné)
predictions = algorithme.predict(X_VALIDATION)

# Score sur le jeu d'apprentissage
precision_apprentissage = algorithme.score(X_APPRENTISSAGE, Y_APPRENTISSAGE)

# Score sur le jeu de validation
precision_validation = r2_score(Y_VALIDATION, predictions)

print("Précision sur le jeu d'apprentissage :", precision_apprentissage)
print("Précision sur le jeu de validation   :", precision_validation)


# Prédictions sur le jeu de validation (avec le dernier modèle entraîné)
predictions = algorithme.predict(X_VALIDATION)

# Score sur le jeu d'apprentissage
precision_apprentissage = algorithme.score(X_APPRENTISSAGE, Y_APPRENTISSAGE)

# Score sur le jeu de validation
precision_validation = r2_score(Y_VALIDATION, predictions)

print("Précision sur le jeu d'apprentissage :", precision_apprentissage)
print("Précision sur le jeu de validation   :", precision_validation)


In [ ]:
import os
import joblib

# On s'assure que le dossier 'modele' existe
os.makedirs("modele", exist_ok=True)

fichier = "modele/modele_pokemon.mod"
joblib.dump(algorithme, fichier)

print("Modèle sauvegardé dans :", fichier)


In [ ]:
dataset.columns

Index(['Unnamed: 0', 'NUMERO', 'NOM', 'TYPE_1', 'TYPE_2', 'POINTS_DE_VIE',
       'NIVEAU_ATTAQUE', 'NIVEAU_DEFENSE', 'NIVEAU_ATTAQUE_SPECIALE',
       'NIVEAU_DEFENSE_SPECIALE', 'VITESSE', 'NOMBRE_GENERATIONS',
       'LEGENDAIRE', 'NBR_COMBATS', 'NBR_VICTOIRES',
       'POURCENTAGE_DE_VICTOIRES'],
      dtype='object')